Add noise (not done)

In [ ]:
bands_snr = [128, 201, 243, 228, 74, 275, 110, 880, 838, 802, 754, 750, 910, 1087, 586, 516, 167, 57, 250,
        None, None, None, None, None, None, 150, None, None, None, None, None, None, None, None, None, None]

bands_netd = [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None,
         None, None, 0.05, 0.2, 0.07, 0.07, 0.25, 0.25, None, 0.25, 0.25, 0.05, 0.25, 0.05, 0.05, 0.25, 0.25, 0.25, 0.35]


snr_netd = zip(bands_snr, bands_netd)
all_bands = data_ir_vis[:,1:30]
print(all_bands[0,:])

print(all_bands.shape)

for band_i, (snr, netd) in enumerate(snr_netd):
    if snr != None:
        avg_squared_signal_pwr = np.mean(np.square(all_bands[:, band_i]))
        noise_std = np.sqrt(avg_squared_signal_pwr / snr)
    else:
        noise_std = netd

    all_bands[:, band_i] += np.random.normal(0, noise_std, all_bands[:, band_i].shape)

In [ ]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    # Define the binary cross-entropy loss
    bce_loss = K.binary_crossentropy(y_true, y_pred)
    
    # Define a penalty for false negatives
    # Adjust the value of alpha based on the importance of reducing false negatives
    alpha = 0.5  # Example value, you can adjust as needed
    false_negative_penalty = alpha * K.sum(y_true * K.cast(K.greater(y_pred, 0.5), 'float32'), axis=-1)
    
    # Add the penalty to the loss
    total_loss = bce_loss + false_negative_penalty
    
    return total_loss

In [ ]:
if USE_FIFTY_FIFTY:

    #data_ir_vis, data_test = train_test_split(data_ir_vis, test_size=0.05, random_state=1)

    low_vis = data_ir_vis[np.where(data_ir_vis[:,30] < low_vis_threshold)[0]]

    #raw_data_ir = pd.read_csv('fog_dataset/' + 'fog_data_ir_clear.dat').to_numpy()
    #raw_data_vis = pd.read_csv('fog_dataset/' + 'fog_data_vis_clear.dat').to_numpy()
    #data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])

    
    
    #data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

    #tmp_data = np.hstack([data_ir, data_vis])
    #tmp_data = tmp_data[:,:-1] 
    #tmp_data = tmp_data.astype(np.float32)

    high_vis_i = np.where(data_ir_vis[:,30] >= low_vis_threshold)[0]      #tmp_data[:,30]
    if len(low_vis) > len(high_vis_i):
       print('Not enough high vis data for 50/50')
    r_high_vis_i = np.random.choice(high_vis_i, len(low_vis), replace=False)
    fifty_fifty = np.vstack([low_vis, data_ir_vis[r_high_vis_i]])       #tmp_data[r_high_vis_i]]
    data_ir_vis = fifty_fifty

In [1]:
y_train_label = np.zeros_like(y_train, dtype=int)
y_val_label = np.zeros_like(y_val, dtype=int)
y_test_label = np.zeros_like(y_test, dtype=int)

for i in np.where(target_scaler.inverse_transform(y_train) < low_vis_threshold)[0]:
   y_train_label[i] = 1

for i in np.where(target_scaler.inverse_transform(y_val) < low_vis_threshold)[0]:
   y_val_label[i] = 1

for i in np.where(target_scaler.inverse_transform(y_test) < low_vis_threshold)[0]:
   y_test_label[i] = 1

NameError: name 'np' is not defined

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



USE_MIXED = False
USE_PRESSURE_TEMP = False
noise_std_dev = 1e-3


if USE_MIXED:
    filenames_vis = ['fog_data_vis_clear.dat', 'fog_data_vis_ice.dat', 'fog_data_vis_mixed.dat', 'fog_data_vis_water.dat']
    filenames_ir = ['fog_data_ir_clear.dat', 'fog_data_ir_ice.dat', 'fog_data_ir_mixed.dat', 'fog_data_ir_water.dat']
else:
    filenames_vis = ['fog_data_vis_clear.dat', 'fog_data_vis_ice.dat', 'fog_data_vis_water.dat']
    filenames_ir = ['fog_data_ir_clear.dat', 'fog_data_ir_ice.dat', 'fog_data_ir_water.dat']

target_i = 30 # Visibility
if USE_PRESSURE_TEMP:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,36))) # 13 ir bands, 10 vis bands, 3 angles, pressure, temp
else:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,34)))



data_ir_vis = []

for file_ir, file_vis in zip(filenames_ir, filenames_vis):
  raw_data_ir = pd.read_csv('fog_dataset/' + file_ir).to_numpy()
  raw_data_vis = pd.read_csv('fog_dataset/' + file_vis).to_numpy()

  data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])
  data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

  if len(data_ir_vis) == 0:
    data_ir_vis = np.hstack([data_ir, data_vis])
  else:
    data_ir_vis = np.vstack([data_ir_vis, np.hstack([data_ir, data_vis])])


# Remove surface description to convert to float
data_ir_vis = data_ir_vis[:,:-1] 
data_ir_vis = data_ir_vis.astype(np.double)

nan_i = np.where(np.isnan(data_ir_vis))[0]
data_ir_vis = np.delete(data_ir_vis, nan_i, axis=0)



X = np.zeros((data_ir_vis.shape[0], len(features_i)))
y = np.zeros(data_ir_vis.shape[0])

for i in range(len(data_ir_vis)):
  X[i] = data_ir_vis[i, features_i]
  y[i] = data_ir_vis[i,target_i]

# 19 Data points contains nan. Not sure why. Band 32 culprit, 8th feature
nan_i = np.where(np.isnan(X))[0]
print(f'nani: {nan_i}')
X = np.delete(X, nan_i, axis=0)
y = np.delete(y, nan_i, axis=0)


# Scaling features
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

X_scaled_noisy = X_scaled + np.random.normal(0, noise_std_dev, X_scaled.shape)

# Scaling target
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.reshape(-1, 1))


# First split: Separate out a test set (5% of the original dataset)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled_noisy, y_scaled, test_size=(1/20), random_state=16)

# Second split: Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(1/19), random_state=16)

In [ ]:
import numpy as np
import tensorflow as tf
from keras import layers, models

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        assert embed_dim % num_heads == 0
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs, training):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=True):
        attn_output = self.att(inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Define your existing features_i, num_features, and other variables
num_features = len(features_i)

# Define your Transformer parameters
num_layers = 2
input_shape = (100, num_features)  # Example input shape: 100 timesteps, num_features features
embed_dim = 32
num_heads = 2
ff_dim = 32
output_dim = 1  # Output dimension, adjust as needed
rate = 0.1  # Dropout rate

# Build the Transformer-based model
def build_transformer_model(num_layers, input_shape, embed_dim, num_heads, ff_dim, output_dim, rate=0.1):
    inputs = layers.Input(shape=input_shape)
    x = inputs
    for _ in range(num_layers):
        x = TransformerEncoder(embed_dim, num_heads, ff_dim, rate)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(rate)(x)
    outputs = layers.Dense(output_dim, activation="sigmoid")(x)
    return models.Model(inputs, outputs)

# Build the Transformer-based model
transformer_model = build_transformer_model(num_layers, input_shape, embed_dim, num_heads, ff_dim, output_dim, rate)

# Concatenate the output of the Transformer model with additional features if needed
combined_input = layers.concatenate([transformer_model.output, ff_model.output])

# Add additional dense layers or any other layers as needed
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(combined_input)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.2)(x)

# Output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the final model
final_model = models.Model(inputs=[transformer_model.input, ff_model.input], outputs=outputs)

# Compile the final model
final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow import keras
from keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # parametreleri
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        # batch-layer
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TabTransformer(keras.Model):

    def __init__(self, 
            categories,
            num_continuous,
            dim,
            dim_out,
            depth,
            heads,
            attn_dropout,
            ff_dropout,
            mlp_hidden,
            normalize_continuous = True):
        """TabTrasformer model constructor
        Args:
            categories (:obj:`list`): list of integers denoting the number of 
                classes for a categorical feature.
            num_continuous (int): number of categorical features
            dim (int): dimension of each embedding layer output, also transformer dimension
            dim_out (int): output dimension of the model
            depth (int): number of transformers to stack
            heads (int): number of attention heads
            attn_dropout (float): dropout to use in attention layer of transformers
            ff_dropout (float): dropout to use in feed-forward layer of transformers
            mlp_hidden (:obj:`list`): list of tuples, indicating the size of the mlp layers and
                their activation functions
            normalize_continuous (boolean, optional): whether the continuous features are normalized
                before MLP layers, True by default
        """
        super(TabTransformer, self).__init__()

        # --> continuous inputs
        self.normalize_continuous = normalize_continuous
        if normalize_continuous:
            self.continuous_normalization = layers.LayerNormalization()

        # --> categorical inputs

        # embedding
        self.embedding_layers = []
        for number_of_classes in categories:
            self.embedding_layers.append(layers.Embedding(input_dim = number_of_classes, output_dim = dim))

        # concatenation
        self.embedded_concatenation = layers.Concatenate(axis=1)

        # adding transformers
        self.transformers = []
        for _ in range(depth):
            self.transformers.append(TransformerBlock(dim, heads, dim))
        self.flatten_transformer_output = layers.Flatten()

        # --> MLP
        self.pre_mlp_concatenation = layers.Concatenate()

        # mlp layers
        self.mlp_layers = []
        for size, activation in mlp_hidden:
            self.mlp_layers.append(layers.Dense(size, activation=activation))

        self.output_layer = layers.Dense(dim_out)

    def call(self, inputs):
        continuous_inputs  = inputs[0]
        categorical_inputs = inputs[1:]
        
        # --> continuous
        if self.normalize_continuous:
            continuous_inputs = self.continuous_normalization(continuous_inputs)

        # --> categorical
        #embedding_outputs = []
        # for categorical_input, embedding_layer in zip(categorical_inputs, self.embedding_layers):
        #     embedding_outputs.append(embedding_layer(categorical_input))
        # categorical_inputs = self.embedded_concatenation(embedding_outputs)
        embedding_outputs = tf.keras.layers.Lambda(lambda x: [layer(x_i) for x_i, layer in zip(x, self.embedding_layers)])(categorical_inputs)
        categorical_inputs = self.embedded_concatenation(embedding_outputs)

        for transformer in self.transformers:
            categorical_inputs = transformer(categorical_inputs)
        contextual_embedding = self.flatten_transformer_output(categorical_inputs)

        # --> MLP
        mlp_input = self.pre_mlp_concatenation([continuous_inputs, contextual_embedding])
        for mlp_layer in self.mlp_layers:
            mlp_input = mlp_layer(mlp_input)

        return self.output_layer(mlp_input)


In [ ]:
from tensorflow import keras

# Define your model instance
tabTransformer = TabTransformer(
    categories=[],        # No categorical features
    num_continuous=26,    # Number of numerical features
    dim=16,               # Embedding/transformer dimension
    dim_out=1,            # Dimension of the model output (1 for binary classification)
    depth=6,              # Number of transformer layers in the stack
    heads=8,              # Number of attention heads
    attn_dropout=0.1,     # Attention layer dropout in transformers
    ff_dropout=0.1,       # Feed-forward layer dropout in transformers
    mlp_hidden=[(32, 'relu'), (16, 'relu')]  # MLP layer dimensions and activations
)


# Compile the model
tabTransformer.compile(optimizer='adam', 
              loss='mse',  # Example loss, use appropriate loss function for your task
              metrics=['mae'])  # Example metric, use appropriate metrics for your task

# Fit the model to your training data
tabTransformer.fit(training_data_classes, epochs = 150, validation_data=(X_val_noisy, y_val_label))


**Load data with TEST_FRACTION Flag**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

# Cloud type index = 37
# clear = 0
# stratus continental = 1
# stratus maritime = 2
# cumulus continental clean = 3
# cumulus continental polluted = 4
# cumulus maritime = 5
# cirrus = 6
# cirrus + water clouds (mixed) >= 7

USE_MIXED = True
USE_PRESSURE_TEMP = False
USE_SPECIFIC_CLOUDS = False
LOW_VIS_FRAC = False
low_vis_frac_train = 0.05
low_vis_threshold = 10000 #50000
cloud_types = [6] #0,1,2 ::: 0, 3, 4
noise_level = 0.01 # % of max
dnn_batch_size = 64

TEST_FRACTION = False

if USE_MIXED:
    filenames_vis = ['fog_data2_vis_clear.dat', 'fog_data2_vis_ice.dat', 'fog_data2_vis_mixed.dat', 'fog_data2_vis_water.dat']
    filenames_ir = ['fog_data2_ir_clear.dat', 'fog_data2_ir_ice.dat', 'fog_data2_ir_mixed.dat', 'fog_data2_ir_water.dat']
else:
    filenames_vis = ['fog_data2_vis_clear.dat', 'fog_data2_vis_ice.dat', 'fog_data2_vis_water.dat']
    filenames_ir = ['fog_data2_ir_clear.dat', 'fog_data2_ir_ice.dat', 'fog_data2_ir_water.dat']

target_i = 30 # Visibility index
if USE_PRESSURE_TEMP:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,36))) # 13 ir bands, 10 vis bands, 3 angles, pressure, temp
else:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,34)))

# Combine .dat files into one numpy array
data_ir_vis = []
for file_ir, file_vis in zip(filenames_ir, filenames_vis):
  raw_data_ir = pd.read_csv('fog_dataset2/' + file_ir).to_numpy()
  raw_data_vis = pd.read_csv('fog_dataset2/' + file_vis).to_numpy()

  data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])
  data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

  if len(data_ir_vis) == 0:
    data_ir_vis = np.hstack([data_ir, data_vis])
  else:
    data_ir_vis = np.vstack([data_ir_vis, np.hstack([data_ir, data_vis])])

# Remove surface description to convert to float
data_ir_vis = data_ir_vis[:,:-1] 
data_ir_vis = data_ir_vis.astype(np.float32)

# 19 Data points contains nan. Not sure why. Band 32 culprit, 8th feature
nan_i = np.where(np.isnan(data_ir_vis))[0]
data_ir_vis = np.delete(data_ir_vis, nan_i, axis=0)

# Cloud type index = 37
if USE_SPECIFIC_CLOUDS:
   data_ir_vis = data_ir_vis[np.isin(data_ir_vis[:, 37], cloud_types)]


X = data_ir_vis[:, features_i]
y = data_ir_vis[:,target_i]

noise_std_devs = np.zeros(features_i.shape)

for i, feature_i in enumerate(features_i):
    noise_std = (np.max(data_ir_vis[:,feature_i]) - np.min(data_ir_vis[:,feature_i])) * noise_level # 1% of mean or max??
    noise_std_devs[i] = noise_std

# Scaling features and noise standard deviations
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)
noise_stds_scaled = np.divide(noise_std_devs, feature_scaler.scale_)

# Scaling target
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.reshape(-1, 1))


if not TEST_FRACTION:
    # First split: Separate out a test set (5% of the original dataset)
    X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y_scaled, test_size=(1/20))


if LOW_VIS_FRAC:
    if not TEST_FRACTION:
        low_vis_i = np.where(target_scaler.inverse_transform(y_temp) < low_vis_threshold)[0]
        high_vis_i = np.where(target_scaler.inverse_transform(y_temp) >= low_vis_threshold)[0]
    else:
        low_vis_i = np.where(target_scaler.inverse_transform(y_scaled) < low_vis_threshold)[0]
        high_vis_i = np.where(target_scaler.inverse_transform(y_scaled) >= low_vis_threshold)[0]


    n_high_vis = int(len(low_vis_i) / low_vis_frac_train * (1 - low_vis_frac_train))
    print(f'real frac: {len(low_vis_i)/len(y_scaled)}')
    if n_high_vis > len(high_vis_i):
        n_high_vis = len(high_vis_i)
        n_low_vis = int(low_vis_frac_train * n_high_vis)
        r_low_vis_i = np.random.choice(low_vis_i, n_low_vis, replace=False)
        low_vis_i = r_low_vis_i
    print(f'n_low_vis: {len(low_vis_i)}. n_high_vis: {n_high_vis}')
    r_high_vis_i = np.random.choice(high_vis_i, n_high_vis, replace=False)
    if not TEST_FRACTION:
        X_temp = np.vstack([X_temp[low_vis_i], X_temp[r_high_vis_i]])
        y_temp = np.vstack([y_temp[low_vis_i], y_temp[r_high_vis_i]])
    else:
        X_temp = np.vstack([X_scaled[low_vis_i], X_scaled[r_high_vis_i]])
        y_temp = np.vstack([y_scaled[low_vis_i], y_scaled[r_high_vis_i]])


if TEST_FRACTION:
    X_temp, X_test, y_temp, y_test = train_test_split(X_temp, y_temp, test_size=(1/19))
  

# Second split: Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(1/19))

# Create noisy data
X_train_noisy = X_train + np.multiply(np.random.normal(0, 1, X_train.shape), noise_stds_scaled)
X_val_noisy = X_val + np.multiply(np.random.normal(0, 1, X_val.shape), noise_stds_scaled)
X_test_noisy = X_test + np.multiply(np.random.normal(0, 1, X_test.shape), noise_stds_scaled)

# Create class labels
y_train_label = np.where(target_scaler.inverse_transform(y_train) < low_vis_threshold, 1, 0)
y_val_label = np.where(target_scaler.inverse_transform(y_val) < low_vis_threshold, 1, 0)
y_test_label = np.where(target_scaler.inverse_transform(y_test) < low_vis_threshold, 1, 0)

# Create training data for DNN that adds noise to batches
training_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size=dnn_batch_size)
training_data_classes = tf.data.Dataset.from_tensor_slices((X_train, y_train_label)).batch(batch_size=dnn_batch_size)

def noise_map(features, labels):
    noise = tf.random.normal(shape=tf.shape(features))
    noise = noise * noise_stds_scaled
    noisy_features = features + noise
    return noisy_features, labels

training_data = training_data.map(noise_map)
training_data_classes = training_data_classes.map(noise_map)
